In [1]:
from typing import Union, List, Dict, Any, cast

import torch
import torch.nn as nn

In [2]:
class VGG(nn.Module):
    def __init__(
        self, features: nn.Module, num_classes: int = 10, init_weights: bool = True, dropout: float = 0.5
    ) -> None:
        super().__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [3]:
def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
    #    print(count)
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    #    count+=1
    return nn.Sequential(*layers)

In [4]:
cfgs: Dict[str, List[Union[str, int]]] = {
    "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
}

In [5]:
vgg11Feature = cfgs["A"]
vgg13Feature = cfgs["B"]
vgg16Feature = cfgs["D"]
vgg19Feature = cfgs["E"]

vgg11Index = [0, 2, 4,5, 7,8, 10,11]
vgg13Index = [0,1, 3,4, 6,7, 9,10, 12,13]
vgg16Index = [0,1, 3,4, 6,7,8, 10,11,12, 14,15,16]
vgg19Index = [0,1, 3,4, 6,7,8,9, 11,12,13,14, 16,17,18,19]
print(vgg11Feature)

[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']


In [6]:
pruneList = [2,4,8,8,16,16,16,16]
count = 0
for i in vgg11Index:
    vgg11Feature[i] = (cast(int,vgg11Feature[i])-pruneList[count])
    count+=1
for i in vgg11Index:
    print(f'index [i] value = {vgg11Feature[i]}')

index [i] value = 62
index [i] value = 124
index [i] value = 248
index [i] value = 248
index [i] value = 496
index [i] value = 496
index [i] value = 496
index [i] value = 496


In [7]:
print(vgg11Feature)

[62, 'M', 124, 'M', 248, 248, 'M', 496, 496, 'M', 496, 496, 'M']


In [8]:
def _vgg(arch: str, cfg: str, batch_norm: bool, pretrained: bool, progress: bool, **kwargs: Any) -> VGG:
    if pretrained:
        kwargs["init_weights"] = False
    
    #Calling the constructer here
    model = VGG(make_layers(vgg11Feature, batch_norm=batch_norm), **kwargs)
    
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
    return model

In [9]:
def vgg11(pretrained: bool = False, progress: bool = True, **kwargs: Any):
    return _vgg("vgg11", "A", False, pretrained, progress, **kwargs)

In [10]:
vggnet = vgg11()
print(vggnet)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 62, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(62, 124, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(124, 248, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(248, 248, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(248, 496, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(496, 496, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [11]:
def deepCopy(newModel,oldModel):
    return newModel